Open in Colab: <a href="https://colab.research.google.com/github/kaicarver/Intent_Classification/blob/master/Intent_classification_final.ipynb" target="_parent">github/kaicarver/Intent_Classification<img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.lancaster import LancasterStemmer
import nltk
import re
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Dense, LSTM, Bidirectional, Embedding, Dropout
from keras.callbacks import ModelCheckpoint


Using TensorFlow backend.


In [21]:
print("hello")

hello


In [2]:
def load_dataset(filename):
  df = pd.read_csv(filename, encoding = "utf8", names = ["Sentence", "Intent"])
  print(df.head())
  intent = df["Intent"]
  unique_intent = list(set(intent))
  sentences = list(df["Sentence"])
  
  return (intent, unique_intent, sentences)
  


In [3]:
#dataset_filename = "Dataset.csv"
dataset_filename = "posos.csv"
intent, unique_intent, sentences = load_dataset(dataset_filename)

                                            Sentence  Intent
0  bonjour,  je m suis trompé de forum pour ma qu...      28
1  est ce que le motilium me soulagera contre les...      31
2  mon médecin m'a prescrit adenyl. au 2ème cache...      28
3  Est-ce qu'il existe une forme adaptée aux enfa...      44
4  mon  medecin  me soigne  pour  une  rhino  pha...      31


In [4]:
print(sentences[:5])

["bonjour,  je m suis trompé de forum pour ma question alors je la repose ici. je pris pour la première fois hier du paroxétine et ce matin c'est une catastrophe. picotement dasn tous le corps annonciateur de sueur froide très très massive et de vomissement. j'en suis à deux crises depuis 5 heure du mat. la cela semble passer mes mes mains reste moites et chaude estce normal pour la première fois merci a tous", 'est ce que le motilium me soulagera contre les nausées?', "mon médecin m'a prescrit adenyl. au 2ème cachet des maux de tête terribles et au 3ème palpitations, sueurs froides, chaleur intense dans la tête, tremblements, fourmillements dans la lèvre supérieure, difficultés à respirer.. dès l'arrêt du médicament tous les symptômes ont disparu. cela est-il déjà arrivé à quelqu'un??", "Est-ce qu'il existe une forme adaptée aux enfant de 5ans du Micropakine ?", "mon  medecin  me soigne  pour  une  rhino  pharingite  et  m'a  prescrit du amoxicilline comme  anti  biotique. Est-ce vrai

In [5]:
nltk.download("stopwords")
nltk.download("punkt")

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/rcarver/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/rcarver/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
#define stemmer
stemmer = LancasterStemmer()

In [7]:
def cleaning(sentences):
  words = []
  for s in sentences:
    clean = re.sub(r'[^ a-z A-Z 0-9]', " ", s)
    w = word_tokenize(clean)
    #stemming
    words.append([i.lower() for i in w])
    
  return words  

In [8]:
cleaned_words = cleaning(sentences)
print(len(cleaned_words))
print(cleaned_words[:2])  
  


8028
[['bonjour', 'je', 'm', 'suis', 'tromp', 'de', 'forum', 'pour', 'ma', 'question', 'alors', 'je', 'la', 'repose', 'ici', 'je', 'pris', 'pour', 'la', 'premi', 're', 'fois', 'hier', 'du', 'parox', 'tine', 'et', 'ce', 'matin', 'c', 'est', 'une', 'catastrophe', 'picotement', 'dasn', 'tous', 'le', 'corps', 'annonciateur', 'de', 'sueur', 'froide', 'tr', 's', 'tr', 's', 'massive', 'et', 'de', 'vomissement', 'j', 'en', 'suis', 'deux', 'crises', 'depuis', '5', 'heure', 'du', 'mat', 'la', 'cela', 'semble', 'passer', 'mes', 'mes', 'mains', 'reste', 'moites', 'et', 'chaude', 'estce', 'normal', 'pour', 'la', 'premi', 're', 'fois', 'merci', 'a', 'tous'], ['est', 'ce', 'que', 'le', 'motilium', 'me', 'soulagera', 'contre', 'les', 'naus', 'es']]


In [9]:
def create_tokenizer(words, filters = '!"#$%&()*+,-./:;<=>?@[\]^_`{|}~'):
  token = Tokenizer(filters = filters)
  token.fit_on_texts(words)
  return token

In [10]:
def max_length(words):
  return(len(max(words, key = len)))
  

In [11]:
word_tokenizer = create_tokenizer(cleaned_words)
vocab_size = len(word_tokenizer.word_index) + 1
max_length = max_length(cleaned_words)

print("Vocab Size = %d and Maximum length = %d" % (vocab_size, max_length))

Vocab Size = 9346 and Maximum length = 778


In [12]:
def encoding_doc(token, words):
  return(token.texts_to_sequences(words))

In [13]:
encoded_doc = encoding_doc(word_tokenizer, cleaned_words)

In [14]:
def padding_doc(encoded_doc, max_length):
  return(pad_sequences(encoded_doc, maxlen = max_length, padding = "post"))

In [15]:
padded_doc = padding_doc(encoded_doc, max_length)

In [16]:
padded_doc[:5]

array([[122,   2,  18, ...,   0,   0,   0],
       [  3,  12,   8, ...,   0,   0,   0],
       [ 35,  18, 141, ...,   0,   0,   0],
       [  3,  12,  43, ...,   0,   0,   0],
       [ 35, 281,  24, ...,   0,   0,   0]], dtype=int32)

In [20]:
print("Shape of padded docs = ",padded_doc.shape)
type(unique_intent)


Shape of padded docs =  (8028, 778)


list

In [18]:
#tokenizer with filter changed
output_tokenizer = create_tokenizer(unique_intent, filters = '!"#$%&()*+,-/:;<=>?@[\]^`{|}~')


AttributeError: 'int' object has no attribute 'lower'

In [ ]:
output_tokenizer.word_index

In [ ]:
encoded_output = encoding_doc(output_tokenizer, intent)

In [ ]:
encoded_output = np.array(encoded_output).reshape(len(encoded_output), 1)

In [ ]:
encoded_output.shape

In [ ]:
def one_hot(encode):
  o = OneHotEncoder(sparse = False)
  return(o.fit_transform(encode))

In [ ]:
output_one_hot = one_hot(encoded_output)

In [ ]:
output_one_hot.shape

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train_X, val_X, train_Y, val_Y = train_test_split(padded_doc, output_one_hot, shuffle = True, test_size = 0.2)


In [ ]:
print("Shape of train_X = %s and train_Y = %s" % (train_X.shape, train_Y.shape))
print("Shape of val_X = %s and val_Y = %s" % (val_X.shape, val_Y.shape))

In [ ]:
def create_model(vocab_size, max_length):
  model = Sequential()
  model.add(Embedding(vocab_size, 128, input_length = max_length, trainable = False))
  model.add(Bidirectional(LSTM(128)))
#   model.add(LSTM(128))
  model.add(Dense(32, activation = "relu"))
  model.add(Dropout(0.5))
  model.add(Dense(21, activation = "softmax"))
  
  return model

In [ ]:
model = create_model(vocab_size, max_length)

model.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])
model.summary()

In [ ]:
filename = 'model.h5'
checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

hist = model.fit(train_X, train_Y, epochs = 100, batch_size = 32, validation_data = (val_X, val_Y), callbacks = [checkpoint])

In [ ]:
 model = load_model("model.h5")

In [ ]:
def predictions(text):
  clean = re.sub(r'[^ a-z A-Z 0-9]', " ", text)
  test_word = word_tokenize(clean)
  test_word = [w.lower() for w in test_word]
  test_ls = word_tokenizer.texts_to_sequences(test_word)
  print(test_word)
  #Check for unknown words
  if [] in test_ls:
    test_ls = list(filter(None, test_ls))
    
  test_ls = np.array(test_ls).reshape(1, len(test_ls))
 
  x = padding_doc(test_ls, max_length)
  
  pred = model.predict_proba(x)
  
  
  return pred


  

In [ ]:
def get_final_output(pred, classes):
  predictions = pred[0]
 
  classes = np.array(classes)
  ids = np.argsort(-predictions)
  classes = classes[ids]
  predictions = -np.sort(-predictions)
 
  for i in range(pred.shape[1]):
    print("%s has confidence = %s" % (classes[i], (predictions[i])))



In [ ]:
text = "Can you help me?"
pred = predictions(text)
get_final_output(pred, unique_intent)